In [1]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM ,Dense ,Bidirectional,Dropout ,Embedding
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import re
import string
import nltk

### Etract the data

In [2]:

data = pd.read_csv("./data/Shakespeare_data.csv")
data.head()
data = data["PlayerLine"]
data = data.to_list()
print(len(data))

111396


### Clean the data

In [3]:
def Clean_data(texts):
    punc = string.punctuation
    cleaned_data = []
    for line in texts :
        clean_text = [word.lower() for word in line if word not in punc]
        clean_text = "".join(clean_text)
        cleaned_data.append(clean_text)
    return cleaned_data

clean_data =Clean_data(data)
clean_data[100]

'which makes him prune himself and bristle up'

### Split the data 

In [4]:
train_split = int(len(data)*0.9)
train_data =clean_data[:train_split]
test_data =clean_data[train_split:]
print("The train data is ",len(train_data),"\n","The test data is ",len(test_data))


The train data is  100256 
 The test data is  11140


### Process the data 


In [5]:

# Assuming 'data' is your list of strings
train_data = clean_data[:train_split]

print(train_data[1])  # Test printing the first element after processing


scene i london the palace


In [11]:
### our tokenizer 
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data)
print(len(tokenizer.word_index))
total_word = len(tokenizer.word_index)+1

26029


In [13]:
max_len =max([len(i) for i in train_data])
max_len

987

In [8]:

def get_sequence_of_tokens(corpus):
    ## tokenization
    corpus = corpus[:7000]
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words
text,total_word =  get_sequence_of_tokens(train_data)



In [19]:
tokenizer.texts_to_sequences(["hey","you"])

[[3512], [7]]

In [9]:
import pickle
# Save the tokenizer to a file using pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [20]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = tf.keras.utils.to_categorical(label, num_classes=total_word)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(text)
print(max_sequence_len)
predictors.shape, label.shape

34


((45584, 33), (45584, 26030))

In [66]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = tf.keras.Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(512))
    model.add(Dropout(0.4))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_word)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 33, 10)            260300    
                                                                 
 lstm (LSTM)                 (None, 512)               1071104   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 26030)             13353390  
                                                                 
Total params: 14,684,794
Trainable params: 14,684,794
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.compile(optimizer="adam",metrics=["accuracy"],loss="categorical_crossentropy")

In [71]:
histroy = model.fit(predictors,label,epochs=100)

Epoch 1/100
1425/1425 [==============================] - 413s 288ms/step - loss: 7.0529 - accuracy: 0.0426
Epoch 2/100
1425/1425 [==============================] - 368s 258ms/step - loss: 6.6044 - accuracy: 0.0532
Epoch 3/100
1425/1425 [==============================] - 350s 246ms/step - loss: 6.4259 - accuracy: 0.0567
Epoch 4/100
1425/1425 [==============================] - 340s 239ms/step - loss: 6.2502 - accuracy: 0.0657
Epoch 5/100
1425/1425 [==============================] - 372s 261ms/step - loss: 6.0711 - accuracy: 0.0766
Epoch 6/100
1425/1425 [==============================] - 409s 287ms/step - loss: 5.8777 - accuracy: 0.0819
Epoch 7/100
1425/1425 [==============================] - 386s 271ms/step - loss: 5.6823 - accuracy: 0.0895
Epoch 8/100
1425/1425 [==============================] - 361s 253ms/step - loss: 5.4566 - accuracy: 0.0971
Epoch 9/100
1425/1425 [==============================] - 380s 267ms/step - loss: 5.2203 - accuracy: 0.1051
Epoch 10/100
1425/1425 [=============

KeyboardInterrupt: 

In [72]:
model.save("E:\workstation\\nlp\\projects\\text_generation\\model")

INFO:tensorflow:Assets written to: E:\workstation\nlp\projects\text_generation\model\assets


INFO:tensorflow:Assets written to: E:\workstation\nlp\projects\text_generation\model\assets
